In [5]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Leer el archivo CSV
file_path = '../../datos/Raw/contratos_dnp_2020_v1.csv'
df_contratos_v1 = pd.read_csv(file_path)

df_contratos_v1 = df_contratos_v1[df_contratos_v1['valor_del_contrato'] != 2870993010534]

# Convertir las columnas de fecha al formato datetime
date_columns = ['fecha_de_firma', 'fecha_de_inicio_del_contrato', 'fecha_de_fin_del_contrato']
for col in date_columns:
    df_contratos_v1[col] = pd.to_datetime(df_contratos_v1[col])


# Calcular la duración del contrato en días
df_contratos_v1['duracion_contrato_dias'] = (df_contratos_v1['fecha_de_fin_del_contrato'] - df_contratos_v1['fecha_de_inicio_del_contrato']).dt.days

# Calcular el valor del contrato por día
df_contratos_v1['valor_contrato_por_dia'] = df_contratos_v1['valor_del_contrato'] / df_contratos_v1['duracion_contrato_dias']

# Extraer el año, mes y día de cada fecha
for col in date_columns:
    df_contratos_v1[col + '_year'] = df_contratos_v1[col].dt.year
    df_contratos_v1[col + '_month'] = df_contratos_v1[col].dt.month
    df_contratos_v1[col + '_day'] = df_contratos_v1[col].dt.day

# Definir las columnas numéricas y categóricas
numeric_features = ['valor_del_contrato', 
                    'fecha_de_firma_year', 'fecha_de_firma_month', 'fecha_de_firma_day',
                    'fecha_de_inicio_del_contrato_year', 'fecha_de_inicio_del_contrato_month', 'fecha_de_inicio_del_contrato_day',
                    'fecha_de_fin_del_contrato_year', 'fecha_de_fin_del_contrato_month', 'fecha_de_fin_del_contrato_day']

categorical_features = ['ciudad', 'tipo_de_contrato', 'estado_contrato', 'modalidad_de_contratacion']

# Definir las transformaciones para cada tipo de columna
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar las transformaciones en un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Definir el modelo de clustering
kmeans = KMeans(n_clusters=3, random_state=42)

# Construir el pipeline completo
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('kmeans', kmeans)])

# Entrenar el modelo con los datos
pipeline.fit(df_contratos_v1)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['valor_del_contrato',
                                                   'fecha_de_firma_year',
                                                   'fecha_de_firma_month',
                                                   'fecha_de_firma_day',
                                                   'fecha_de_inicio_del_contrato_year',
                                                   'fecha_de_inicio_del_contrato_month',
                                                   'fecha_de_inicio_del_contra...
                                                   'fecha_de_fin_del_contrato_year',
                                                   'fecha_de_fin_del_contrato_month',
                                                   'fecha_de_fin_del_contrato_day']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['ciudad', 'tipo_de_contrato',
                                                   'estado_contrato',
                                                   'modalidad_de_contratacion'])])),
                ('kmeans', KMeans(n_clusters=3, random_state=42))])

In [6]:

# Clasificar nuevos datos
nuevos_datos = pd.DataFrame({'ciudad': ['Bogotá'],
                             'tipo_de_contrato': ['Prestación de servicios'],
                             'estado_contrato': ['En ejecución'],
                             'modalidad_de_contratacion': ['Contratación directa'],
                             'valor_del_contrato': [5000000000],
                             'fecha_de_firma_year': [2024], 'fecha_de_firma_month': [5], 'fecha_de_firma_day': [18],
                             'fecha_de_inicio_del_contrato_year': [2024], 'fecha_de_inicio_del_contrato_month': [4], 'fecha_de_inicio_del_contrato_day': [18],
                             'fecha_de_fin_del_contrato_year': [2024], 'fecha_de_fin_del_contrato_month': [4], 'fecha_de_fin_del_contrato_day': [18]})

prediccion = pipeline.predict(nuevos_datos)
print("Predicción para los nuevos datos:", prediccion)

Predicción para los nuevos datos: [2]


In [7]:
# Acceder a las etiquetas de clúster asignadas por KMeans
etiquetas_clust = pipeline.named_steps['kmeans'].labels_

# Agregar las etiquetas de clúster al DataFrame original
df_contratos_v1['cluster'] = etiquetas_clust

# Se guarda la nueva base
df_contratos_v1.to_csv('../../datos/Raw/contratos_dnp_2020_cluster.csv')



In [8]:
df_contratos_v1['cluster'].value_counts()

cluster
0    2526
2    2118
1     632
Name: count, dtype: int64

In [ ]:
import matplotlib.pyplot as plt

# Histogramas para características numéricas
for feature in numeric_features:
    plt.figure(figsize=(8, 4))
    for cluster in range(3):  # Cambia 3 por el número de clústeres
        plt.hist(df_contratos_v1[df_contratos_v1['cluster'] == cluster][feature], alpha=0.5, label=f'Cluster {cluster}')
    plt.title(f'Histograma de {feature} por cluster')
    plt.legend()
    plt.show()

# Gráficos de barras para características categóricas
for feature in categorical_features:
    plt.figure(figsize=(8, 4))
    for cluster in range(3):  # Cambia 3 por el número de clústeres
        counts = df_contratos_v1[df_contratos_v1['cluster'] == cluster][feature].value_counts(normalize=True)
        plt.bar(counts.index, counts.values, alpha=0.5, label=f'Cluster {cluster}')
    plt.title(f'Proporción de {feature} por cluster')
    plt.legend()
    plt.show()


In [10]:
# Estadísticas descriptivas por grupo
for cluster in range(3):  # Cambia 3 por el número de clústeres
    print(f"Cluster {cluster}:")
    print(df_contratos_v1[df_contratos_v1['cluster'] == cluster][numeric_features].describe())
    print(df_contratos_v1[df_contratos_v1['cluster'] == cluster][categorical_features].mode())
    print()


Cluster 0:
       valor_del_contrato  fecha_de_firma_year  fecha_de_firma_month  \
count        2.526000e+03          2526.000000           2526.000000   
mean         9.959337e+07          2021.104513              1.382819   
std          1.312875e+08             0.800596              0.846382   
min          0.000000e+00          2020.000000              1.000000   
25%          6.672676e+07          2020.000000              1.000000   
50%          9.486071e+07          2021.000000              1.000000   
75%          1.178426e+08          2022.000000              1.000000   
max          5.730000e+09          2022.000000              8.000000   

       fecha_de_firma_day  fecha_de_inicio_del_contrato_year  \
count         2526.000000                        2521.000000   
mean            17.979810                        2021.103530   
std              7.459301                           0.803756   
min              1.000000                        2018.000000   
25%             13.0

In [90]:
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Leer el archivo CSV
file_path = '../../datos/Raw/contratos_dnp_2020_v1.csv'
df_contratos_v1 = pd.read_csv(file_path)

# Aplicar el filtro
df_contratos_v1 = df_contratos_v1[df_contratos_v1['valor_del_contrato'] != 2870993010534]

# Convertir las columnas de fecha al formato datetime
date_columns = ['fecha_de_firma', 'fecha_de_inicio_del_contrato', 'fecha_de_fin_del_contrato']
for col in date_columns:
    df_contratos_v1[col] = pd.to_datetime(df_contratos_v1[col])

# Calcular la duración del contrato en días
df_contratos_v1['duracion_contrato_dias'] = (df_contratos_v1['fecha_de_fin_del_contrato'] - df_contratos_v1['fecha_de_inicio_del_contrato']).dt.days.fillna(0).replace(0, 1)

# Convertir la columna 'duracion_contrato_dias' a numérica
df_contratos_v1['duracion_contrato_dias'] = df_contratos_v1['duracion_contrato_dias'].astype(int)

# Calcular el valor del contrato por día
df_contratos_v1['valor_contrato_por_dia'] = df_contratos_v1['valor_del_contrato'] / df_contratos_v1['duracion_contrato_dias']

# Extraer el mes de la fecha de firma
df_contratos_v1['fecha_de_firma_month'] = df_contratos_v1['fecha_de_firma'].dt.month

# Definir las columnas numéricas y categóricas
numeric_features = ['valor_del_contrato', 'fecha_de_firma_month', 'duracion_contrato_dias', 'valor_contrato_por_dia']
categorical_features = ['ciudad', 'tipo_de_contrato', 'estado_contrato', 'modalidad_de_contratacion']

# Definir las transformaciones para las características numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Definir las transformaciones para las características categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar las transformaciones en un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Definir el modelo de clustering
kmeans = KMeans(n_clusters=3, random_state=42)

# Construir el pipeline completo
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('kmeans', kmeans)])

# Entrenar el modelo con los datos
pipeline.fit(df_contratos_v1)

# Crear nuevos datos con todas las columnas requeridas
nuevos_datos = pd.DataFrame({'ciudad': ['Bogota'],
                             'tipo_de_contrato': ['Prestación de servicios'],
                             'estado_contrato': ['En ejecución'],
                             'modalidad_de_contratacion': ['Regimen especial'],
                             'fecha_de_firma_month': [1],  # Se debe proporcionar el mes de la fecha de firma
                             'duracion_contrato_dias': [119],
                             'valor_del_contrato': [37724000],  # Asegúrate de que esta columna exista
                             'valor_contrato_por_dia': [317008]})  # Asegúrate de que esta columna exista

# Realizar la predicción
prediccion = pipeline.predict(nuevos_datos)
print("Predicción para los nuevos datos:", prediccion)

# Obtener las etiquetas de cluster asignadas por DBSCAN
etiquetas_clust = pipeline.named_steps['kmeans'].labels_

# Agregar las etiquetas de cluster al DataFrame original
df_contratos_v1['cluster'] = etiquetas_clust

# Guardar el DataFrame con las etiquetas de cluster
df_contratos_v1.to_excel('../../datos/Raw/contratos_dnp_2020_cluster.xlsx', index=False)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['valor_del_contrato',
                                                   'fecha_de_firma_month',
                                                   'duracion_contrato_dias',
                                                   'valor_contrato_por_dia']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['ciudad', 'tipo_de_contrato',
                                                   'estado_contrato',
                                                   'modalidad_de_contratacion'])])),
                ('kmeans', KMeans(n_clusters=3, random_state=42))])

Predicción para los nuevos datos: [0]


In [85]:
df_contratos_v1['cluster'].value_counts()

cluster
0    3488
2    1787
1       1
Name: count, dtype: int64

DBSCAN

In [87]:
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Leer el archivo CSV
file_path = '../../datos/Raw/contratos_dnp_2020_v1.csv'
df_contratos_v1 = pd.read_csv(file_path)

# Aplicar el filtro
df_contratos_v1 = df_contratos_v1[df_contratos_v1['valor_del_contrato'] != 2870993010534]

# Convertir las columnas de fecha al formato datetime
date_columns = ['fecha_de_firma', 'fecha_de_inicio_del_contrato', 'fecha_de_fin_del_contrato']
for col in date_columns:
    df_contratos_v1[col] = pd.to_datetime(df_contratos_v1[col])

# Calcular la duración del contrato en días
df_contratos_v1['duracion_contrato_dias'] = (df_contratos_v1['fecha_de_fin_del_contrato'] - df_contratos_v1['fecha_de_inicio_del_contrato']).dt.days.fillna(0).replace(0, 1)

# Convertir la columna 'duracion_contrato_dias' a numérica
df_contratos_v1['duracion_contrato_dias'] = df_contratos_v1['duracion_contrato_dias'].astype(int)

# Calcular el valor del contrato por día
df_contratos_v1['valor_contrato_por_dia'] = df_contratos_v1['valor_del_contrato'] / df_contratos_v1['duracion_contrato_dias']

# Extraer el mes de la fecha de firma
df_contratos_v1['fecha_de_firma_month'] = df_contratos_v1['fecha_de_firma'].dt.month

# Definir las columnas numéricas y categóricas
numeric_features = ['valor_del_contrato', 'fecha_de_firma_month', 'duracion_contrato_dias', 'valor_contrato_por_dia']
categorical_features = ['ciudad', 'tipo_de_contrato', 'estado_contrato', 'modalidad_de_contratacion']

# Definir las transformaciones para las características numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Definir las transformaciones para las características categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar las transformaciones en un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Definir el modelo de clustering (DBSCAN)
dbscan = DBSCAN(eps=0.8, min_samples=10)  # Puedes ajustar estos parámetros según tus necesidades

# Construir el pipeline completo
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('dbscan', dbscan)])

# Entrenar el modelo con los datos
pipeline.fit(df_contratos_v1)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['valor_del_contrato',
                                                   'fecha_de_firma_month',
                                                   'duracion_contrato_dias',
                                                   'valor_contrato_por_dia']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['ciudad', 'tipo_de_contrato',
                                                   'estado_contrato',
                                                   'modalidad_de_contratacion'])])),
                ('dbscan', DBSCAN(eps=0.8, min_samples=10))])

In [88]:
# Crear nuevos datos con todas las columnas requeridas
nuevos_datos = pd.DataFrame({'ciudad': ['Bogota'],
                             'tipo_de_contrato': ['Prestación de servicios'],
                             'estado_contrato': ['En ejecución'],
                             'modalidad_de_contratacion': ['Regimen especial'],
                             'fecha_de_firma_month': [1],  # Se debe proporcionar el mes de la fecha de firma
                             'duracion_contrato_dias': [119],
                             'valor_del_contrato': [37724000],  # Asegúrate de que esta columna exista
                             'valor_contrato_por_dia': [317008]})  # Asegúrate de que esta columna exista

from sklearn.preprocessing import FunctionTransformer

# Utiliza el preprocesador definido previamente para transformar los nuevos datos
preprocessed_nuevos_datos = preprocessor.transform(nuevos_datos)

# Realizar la predicción con DBSCAN en los nuevos datos preprocesados
prediccion_nuevos_datos = dbscan.fit_predict(preprocessed_nuevos_datos)

# Imprimir las predicciones
print("Predicción para los nuevos datos:", prediccion_nuevos_datos)


Predicción para los nuevos datos: [-1]


In [79]:
# Obtener las etiquetas de los clusters asignadas a los datos de entrenamiento
etiquetas_clust = pipeline.named_steps['dbscan'].labels_

# Agregar las etiquetas de cluster al DataFrame original
df_contratos_v1['cluster'] = etiquetas_clust

# Guardar el DataFrame con las etiquetas de cluster
df_contratos_v1.to_csv('../../datos/Raw/contratos_dnp_2020_cluster.csv', index=False)

df_contratos_v1['cluster'].value_counts()

cluster
 1     2244
 0      873
 5      470
 2      449
-1      392
 3      333
 6      166
 4      139
 8       81
 13      19
 12      18
 14      16
 7       14
 9       14
 10      14
 15      14
 11      10
 16      10
Name: count, dtype: int64